# APWP Analysis

In [ ]:
# standard modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.patches as patches
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
pd.set_option('display.max_rows',999)
pd.set_option('display.max_columns',999)

# pmagpy
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import cartopy.crs as ccrs
import xml.etree.ElementTree as ET
import pygplates as pgp

## Functions

In [ ]:
def create_vgp_FeatureCollection(compilation):
    """
    Loop through poles and produce a pygplates FeatureCollection of multiple VGPs.
    
    Modified from code by Michael G. Tetley.
    
    Parameters
    ----------
    compilation : dataframe
        pole compilation
        
    Returns
    -------
    vpgFeatureCollection : FeatureCollection
        pygplates FeatureCollection of VGPs in compilation
    """
    poleLat = []
    poleLon = []
    poleName = []
    poleSiteLat = []
    poleSiteLon = []
    poleNominalAge = []
    poleA95 = []
    poleAgeLowerLimit = []
    poleAgeUpperLimit = []
    plateID = []

    count = 0

    for i in range(len(compilation)):

        if np.isfinite(compilation['slat'][i]) and np.isfinite(compilation['slon'][i]) and \
           np.isfinite(compilation['age lower'][i]) and np.isfinite(compilation['age upper'][i]):

            poleLat.append(compilation['plat'][i])
            poleLon.append(compilation['plon'][i])

            poleName.append(compilation['name'][i] + ' (' + compilation['grade'][i] + ')')
            poleSiteLat.append(compilation['slat'][i])
            poleSiteLon.append(compilation['slon'][i])
            poleNominalAge.append(compilation['age'][i])
            poleA95.append(compilation['a95'][i])

            poleAgeLowerLimit.append(compilation['age lower'][i])
            poleAgeUpperLimit.append(compilation['age upper'][i])

            plateID.append(compilation['plateID'][i])

            count = count + 1

        # Print if any of the isfinite tests fail
        else:

            print('Bad data for : {}'.format(compilation['name'][i]))


    # Create new GPlates Feature Collection
    vpgFeatureCollection = pgp.FeatureCollection()

    # Create new GPlates feature 'VirtualGeomagneticPole'.
    # Pole lat, pole lon, pole name, and reconstruction plate ID added within PointOnSphere method.
    # Inc, Dec, A95, Age and Sample site lat/lon values to added within 'other_properties' method.

    for j in range(count):

        vgpFeature = pgp.Feature.create_reconstructable_feature(
                     pgp.FeatureType.create_gpml('VirtualGeomagneticPole'),
                     pgp.PointOnSphere([np.float(poleLat[j]), np.float(poleLon[j])]),
                     name = poleName[j],
                     reconstruction_plate_id = int(plateID[j]),
                     other_properties = [(pgp.PropertyName.create_gpml('poleA95'), pgp.XsDouble(np.float64(poleA95[j]))),
                                         (pgp.PropertyName.create_gpml('averageAge'), pgp.XsDouble(np.float64(poleNominalAge[j]))),
                                         (pgp.PropertyName.create_gpml('averageSampleSitePosition'),
                                          pgp.GmlPoint(pgp.PointOnSphere([np.float(poleSiteLat[j]), 
                                                                          np.float(poleSiteLon[j])])))])

        # Add newly created feature to existing Feature Collection
        vpgFeatureCollection.add(vgpFeature)

    return vpgFeatureCollection

In [ ]:
def create_vgp_gpml(vpgFeatureCollection, filename):
    """
    Create a .gpml for a FeatureCollection of VGPs.
    
    Modified from code by Michael G. Tetley.
    
    Parameters
    ----------
    vpgFeatureCollection : FeatureCollection
        pygplates FeatureCollection of VGPs in compilation
        
    filename : string
        path and name for output .gpml
    """
    # Generate GPML output file
    gpmlOutputFile = filename

    # Check for existing output file with same name and remove if found
    if os.path.isfile(filename):
        os.remove(filename)

    # Check to make sure vgpFeatureCollection (feature collection) is not empty before writing to file
    if len(vpgFeatureCollection) != 0:
        outputFeatureCollection = pgp.FeatureCollectionFileFormatRegistry()
        outputFeatureCollection.write(vpgFeatureCollection, filename)

    # Check if new file was created and confirm export
    if os.path.isfile(filename):
        print('Palaeomagnetic pole data successfully exported in GPML format.')

## South China Poles

In [ ]:
compilation_cols = ['name',
                    'terrane',
                    'age',
                    'age upper',
                    'age lower',
                    'plat',
                    'plon',
                    'a95',
                    'f',
                    'slat',
                    'slon',
                    'dec',
                    'inc',
                    'dir_a95',
                    'pole ref',
                    'age ref',
                    'grade',
                    'note',
                    'plateID']

In [ ]:
SChina = pd.DataFrame(columns=compilation_cols)
i = 0

### Yanbian Dikes

* A 5 degree vertical axis rotation is applied to these data in the paper. They are not tilt-corrected (except for one) although they are located within a mobile belt.
* Baked contact test, dike-tilt test, rock magnetism.
* SIMS on dike.

In [ ]:
SChina.loc[i, 'name']    = 'Yanbian Dikes Group A'
SChina.loc[i, 'terrane'] = 'SChina'
SChina.loc[i, 'age']       = 824
SChina.loc[i, 'age upper'] = 824 + 6
SChina.loc[i, 'age lower'] = 824 - 6
SChina.loc[i, 'plat'] = 45.1
SChina.loc[i, 'plon'] = 130.4
SChina.loc[i, 'a95']  = 19.0
SChina.loc[i, 'f'] = 1.0
SChina.loc[i, 'slat'] = 26.886
SChina.loc[i, 'slon'] = 101.546
SChina.loc[i, 'dec']     = np.nan
SChina.loc[i, 'inc']     = np.nan
SChina.loc[i, 'dir_a95'] = np.nan
SChina.loc[i, 'pole ref'] = 'Niu et al. (2016)'
SChina.loc[i, 'age ref']  = 'Niu et al. (2016)'
SChina.loc[i, 'grade'] = 'B'
SChina.loc[i, 'note']  = '-'
SChina.loc[i, 'plateID'] = 602
i = i + 1

### Madiyi Formation

* Inclination correction preferred by authors.
* Reversal test.
* SIMS on tuff.
* Age updated with CA-ID-TIMS in this study.

In [ ]:
SChina.loc[i, 'name']    = 'Madiyi Formation (f=1.0)'
SChina.loc[i, 'terrane'] = 'SChina'
SChina.loc[i, 'age']       = 804.9
SChina.loc[i, 'age upper'] = 804.9 + 0.36
SChina.loc[i, 'age lower'] = 804.9 - 0.36
SChina.loc[i, 'plat'] = 35.3
SChina.loc[i, 'plon'] = 67.9
SChina.loc[i, 'a95']  = np.sqrt(4.7*5.5)
SChina.loc[i, 'f'] = 1.0
SChina.loc[i, 'slat'] = 27.5
SChina.loc[i, 'slon'] = 109.6
SChina.loc[i, 'dec']     = 293.1
SChina.loc[i, 'inc']     = 69.9
SChina.loc[i, 'dir_a95'] = 3.2
SChina.loc[i, 'pole ref'] = 'Xian et al. (2020)'
SChina.loc[i, 'age ref']  = 'this study'
SChina.loc[i, 'grade'] = 'NR'
SChina.loc[i, 'note']  = '-'
SChina.loc[i, 'plateID'] = 602
i = i + 1

In [ ]:
SChina.loc[i, 'name']    = 'Madiyi Formation (f=0.6)'
SChina.loc[i, 'terrane'] = 'SChina'
SChina.loc[i, 'age']       = 804.9
SChina.loc[i, 'age upper'] = 804.9 + 0.36
SChina.loc[i, 'age lower'] = 804.9 - 0.36
SChina.loc[i, 'plat'] = 34.3
SChina.loc[i, 'plon'] = 82.4
SChina.loc[i, 'a95']  = np.sqrt(3.9*3.7)
SChina.loc[i, 'f'] = 0.6
SChina.loc[i, 'slat'] = 27.5
SChina.loc[i, 'slon'] = 109.6
SChina.loc[i, 'dec']     = 293.0
SChina.loc[i, 'inc']     = 77.3
SChina.loc[i, 'dir_a95'] = 2.1
SChina.loc[i, 'pole ref'] = 'Xian et al. (2020)'
SChina.loc[i, 'age ref']  = 'this study'
SChina.loc[i, 'grade'] = 'NR'
SChina.loc[i, 'note']  = '-'
SChina.loc[i, 'plateID'] = 602
i = i + 1

### Chengjiang Formation

* Fold test, reversal test.
* Sample mean - a pole could be calculated from the sites instead. The site mean pole gives roughly the same direction, just with a larger uncertainty.
* SIMS on tuffaceous siltstone.

## Other Poles

In [ ]:
pole_path = '../../../Rodinia_Model/Torsvik-extended/poles/raw-data/'

### Laurentia

#### Compilation

In [ ]:
Laurentia = pd.DataFrame(columns=compilation_cols)

In [ ]:
SwansonHysell_preprint = pd.read_csv(pole_path+'Swanson-Hysell_preprint.csv')
SwansonHysell_preprint.columns

Clean up the age column:

In [ ]:
SwansonHysell_preprint_age_nominal = []
SwansonHysell_preprint_age_upper = []
SwansonHysell_preprint_age_lower = []

for age in SwansonHysell_preprint['age']:
    age_split = age.split('$')
    age_nominal = np.float(age_split[0])
    age_split_upper_lower = age_split[1].split('}')
    age_split_upper = age_split_upper_lower[0].split('+')[1]
    age_split_lower = age_split_upper_lower[1].split('-')[1]
    age_upper = np.float(age_split_upper)
    age_lower = np.float(age_split_lower)
    
    SwansonHysell_preprint_age_nominal.append(age_nominal)
    SwansonHysell_preprint_age_upper.append(age_nominal + age_upper)
    SwansonHysell_preprint_age_lower.append(age_nominal - age_lower)

Assign columns:

In [ ]:
Laurentia['name'] = SwansonHysell_preprint['unit name']
Laurentia['terrane'] = SwansonHysell_preprint['terrane']
Laurentia['age'] = SwansonHysell_preprint_age_nominal
Laurentia['age upper'] = SwansonHysell_preprint_age_upper
Laurentia['age lower'] = SwansonHysell_preprint_age_lower
Laurentia['plat'] = SwansonHysell_preprint['plat']
Laurentia['plon'] = SwansonHysell_preprint['plon']
Laurentia['a95'] = SwansonHysell_preprint['a95']
#Laurentia['f']
Laurentia['slat'] = SwansonHysell_preprint['slat']
Laurentia['slon'] = SwansonHysell_preprint['slon']
#Laurentia['dec']
#Laurentia['inc']
#Laurentia['dir_a95']
Laurentia['pole ref'] = SwansonHysell_preprint['pole reference']
#Laurentia['age ref']
Laurentia['grade'] = SwansonHysell_preprint['Nordic rating']
#Laurentia['note']
#Laurentia['plateID']

Add Adirondack poles:

In [ ]:
i = len(Laurentia)

Laurentia.loc[i, 'name']    = 'Adirondack fayalite granite'
Laurentia.loc[i, 'terrane'] = 'Laurentia'
Laurentia.loc[i, 'age']       = 990
Laurentia.loc[i, 'age upper'] = 990 + 20
Laurentia.loc[i, 'age lower'] = 990 - 20
Laurentia.loc[i, 'plat'] = -28.4
Laurentia.loc[i, 'plon'] = 132.7
Laurentia.loc[i, 'a95']  = 6.9
Laurentia.loc[i, 'f'] = np.nan
Laurentia.loc[i, 'slat'] = 44.0
Laurentia.loc[i, 'slon'] = 285.5
Laurentia.loc[i, 'dec']     = np.nan
Laurentia.loc[i, 'inc']     = np.nan
Laurentia.loc[i, 'dir_a95'] = np.nan
Laurentia.loc[i, 'pole ref'] = 'Brown et al. (2012)'
Laurentia.loc[i, 'age ref']  = '-'
Laurentia.loc[i, 'grade'] = 'NR'
Laurentia.loc[i, 'note']  = '-'
Laurentia.loc[i, 'plateID'] = 101
i = i + 1

Laurentia.loc[i, 'name']    = 'Adirondack metamorphic anorthosites'
Laurentia.loc[i, 'terrane'] = 'Laurentia'
Laurentia.loc[i, 'age']       = 970
Laurentia.loc[i, 'age upper'] = 970 + 20
Laurentia.loc[i, 'age lower'] = 970 - 20
Laurentia.loc[i, 'plat'] = -25.1
Laurentia.loc[i, 'plon'] = 149.0
Laurentia.loc[i, 'a95']  = 11.6
Laurentia.loc[i, 'f'] = np.nan
Laurentia.loc[i, 'slat'] = 44.0
Laurentia.loc[i, 'slon'] = 286.0
Laurentia.loc[i, 'dec']     = np.nan
Laurentia.loc[i, 'inc']     = np.nan
Laurentia.loc[i, 'dir_a95'] = np.nan
Laurentia.loc[i, 'pole ref'] = 'Brown et al. (2012)'
Laurentia.loc[i, 'age ref']  = '-'
Laurentia.loc[i, 'grade'] = 'NR'
Laurentia.loc[i, 'note']  = '-'
Laurentia.loc[i, 'plateID'] = 101
i = i + 1

Laurentia.loc[i, 'name']    = 'Adirondack Microcline gneiss'
Laurentia.loc[i, 'terrane'] = 'Laurentia'
Laurentia.loc[i, 'age']       = 960
Laurentia.loc[i, 'age upper'] = 960 + 20
Laurentia.loc[i, 'age lower'] = 960 - 20
Laurentia.loc[i, 'plat'] = -18.4
Laurentia.loc[i, 'plon'] = 151.1
Laurentia.loc[i, 'a95']  = 10.5
Laurentia.loc[i, 'f'] = np.nan
Laurentia.loc[i, 'slat'] = 44.0
Laurentia.loc[i, 'slon'] = 285.0
Laurentia.loc[i, 'dec']     = np.nan
Laurentia.loc[i, 'inc']     = np.nan
Laurentia.loc[i, 'dir_a95'] = np.nan
Laurentia.loc[i, 'pole ref'] = 'Brown et al. (2012)'
Laurentia.loc[i, 'age ref']  = '-'
Laurentia.loc[i, 'grade'] = 'NR'
Laurentia.loc[i, 'note']  = '-'
Laurentia.loc[i, 'plateID'] = 101
i = i + 1

Assign plateIDs:

In [ ]:
# 101 for most poles
Laurentia['plateID'] = 101

# other poles
Laurentia.loc[Laurentia['name']=='Torridon Group','plateID'] = 303
Laurentia.loc[Laurentia['name']=='Lower Grusdievbreen Formation','plateID'] = 311
Laurentia.loc[Laurentia['name']=='Upper Grusdievbreen Formation','plateID'] = 311
Laurentia.loc[Laurentia['name']=='Svanbergfjellet Formation','plateID'] = 311

Flip polarity of some poles:

In [ ]:
flip_poles = ['Long Range Dykes','Baie des Moutons complex','Callander Alkaline Complex','Catoctin Basalts','Sept-Iles layered intrusion']
for i in Laurentia.index:
    if Laurentia['name'][i] in flip_poles:
        Laurentia.loc[i,'dec'] = (Laurentia['dec'][i] + 180) % 360
        Laurentia.loc[i,'inc'] = -Laurentia['inc'][i]
        Laurentia.loc[i,'plat'] = -Laurentia['plat'][i]
        Laurentia.loc[i,'plon'] = (Laurentia['plon'][i] + 180) % 360

#### Summary

In [ ]:
Laurentia.sort_values('age', ascending=False, inplace=True)
Laurentia = Laurentia[compilation_cols]
Laurentia.reset_index(inplace=True, drop=True)
Laurentia

In [ ]:
Laurentia.to_csv('../Output/Laurentia-poles.csv',index=False)
Laurentia_vpgFeatureCollection = create_vgp_FeatureCollection(Laurentia)
create_vgp_gpml(Laurentia_vpgFeatureCollection, '../Output/Laurentia-poles.gpml')

### India

#### Compilation

In [ ]:
India = pd.DataFrame(columns=compilation_cols)

In [ ]:
Leirubakki_India = pd.read_csv(pole_path+'Leirubakki-India.csv')
Leirubakki_India.columns

Clean up the pole references column:

In [ ]:
Leirubakki_India_pole_ref = []

for i in range(len(Leirubakki_India)):
    pole_ref = str(Leirubakki_India['POLE AUTHORS'][i]) + ' (' + str(Leirubakki_India['YEAR'][i]) + ')'
    Leirubakki_India_pole_ref.append(pole_ref)

Assign columns:

In [ ]:
India['name'] = Leirubakki_India['POLENAME']
India['terrane'] = Leirubakki_India['Terrane']
India['age'] = Leirubakki_India['nominal age']
India['age upper'] = Leirubakki_India['himagage']
India['age lower'] = Leirubakki_India['lomagage']
India['plat'] = Leirubakki_India['PLAT']
India['plon'] = Leirubakki_India['PLON']
India['a95'] = Leirubakki_India['A95']
#India['f']
India['slat'] = Leirubakki_India['SLAT']
India['slon'] = Leirubakki_India['SLON']
India['dec'] = Leirubakki_India['DEC']
India['inc'] = Leirubakki_India['INC']
#India['dir_a95']
India['pole ref'] = Leirubakki_India_pole_ref
#India['age ref']
India['grade'] = Leirubakki_India['Grade']
#India['note']
India['plateID'] = 501

Update age of the Malani Igneous Suite:

In [ ]:
India.loc[India['name']=='Malani Igneous Suite -Comb','age'] = 752
India.loc[India['name']=='Malani Igneous Suite -Comb','age upper'] = 752 + 18
India.loc[India['name']=='Malani Igneous Suite -Comb','age lower'] = 752 - 18

#### Summary

In [ ]:
India.sort_values('age', ascending=False, inplace=True)
India = India[compilation_cols]
India.reset_index(inplace=True, drop=True)
India

In [ ]:
India.to_csv('../Output/India-poles.csv',index=False)
India_vpgFeatureCollection = create_vgp_FeatureCollection(India)
create_vgp_gpml(India_vpgFeatureCollection, '../Output/India-poles.gpml')

### Australia

#### Compilation

In [ ]:
Australia = pd.DataFrame(columns=compilation_cols)

In [ ]:
Leirubakki_Australia = pd.read_csv(pole_path+'Leirubakki-Australia.csv')
Leirubakki_Australia.columns

Clean up the pole references column:

In [ ]:
Leirubakki_Australia_pole_ref = []

for i in range(len(Leirubakki_Australia)):
    pole_ref = str(Leirubakki_Australia['POLE AUTHORS'][i]) + ' (' + str(Leirubakki_Australia['YEAR'][i]) + ')'
    Leirubakki_Australia_pole_ref.append(pole_ref)

Assign columns:

In [ ]:
Australia['name'] = Leirubakki_Australia['POLENAME']
Australia['terrane'] = Leirubakki_Australia['Terrane']
Australia['age'] = Leirubakki_Australia['nominal age']
Australia['age upper'] = Leirubakki_Australia['himagage']
Australia['age lower'] = Leirubakki_Australia['lomagage']
Australia['plat'] = Leirubakki_Australia['PLAT']
Australia['plon'] = Leirubakki_Australia['PLON']
Australia['a95'] = Leirubakki_Australia['A95']
#Australia['f']
Australia['slat'] = Leirubakki_Australia['SLAT']
Australia['slon'] = Leirubakki_Australia['SLON']
Australia['dec'] = Leirubakki_Australia['DEC']
Australia['inc'] = Leirubakki_Australia['INC']
#Australia['dir_a95']
Australia['pole ref'] = Leirubakki_Australia_pole_ref
#Australia['age ref']
Australia['grade'] = Leirubakki_Australia['Grade']
#Australia['note']
#Australia['plateID']

Assign plateIDs:

In [ ]:
# 801 for most poles
Australia['plateID'] = 801

# other poles
Australia.loc[Australia['name']=='Cap Dolomite, Walsh Tillite','plateID'] = 8011
Australia.loc[Australia['name']=='Lower Arumbera and Upper Pertatataka Formations','plateID'] = 8011
Australia.loc[Australia['name']=="Johnny's Creek siltstones",'plateID'] = 8011
Australia.loc[Australia['name']=='Mt.Isa Dolerite Dykes (IAR)','plateID'] = 8011
Australia.loc[Australia['name']=='Alcurra dykes+sills','plateID'] = 8011

#### Summary

In [ ]:
Australia.sort_values('age', ascending=False, inplace=True)
Australia = Australia[compilation_cols]
Australia.reset_index(inplace=True, drop=True)
Australia

In [ ]:
Australia.to_csv('../Output/Australia-poles.csv',index=False)
Australia_vpgFeatureCollection = create_vgp_FeatureCollection(Australia)
create_vgp_gpml(Australia_vpgFeatureCollection, '../Output/Australia-poles.gpml')

### Siberia

#### Compilation

In [ ]:
Siberia = pd.DataFrame(columns=compilation_cols)

In [ ]:
Leirubakki_Siberia = pd.read_csv(pole_path+'Leirubakki-Siberia.csv')
Leirubakki_Siberia.columns

Clean up the pole references column:

In [ ]:
Leirubakki_Siberia_pole_ref = []

for i in range(len(Leirubakki_Siberia)):
    pole_ref = str(Leirubakki_Siberia['POLE AUTHORS'][i]) + ' (' + str(Leirubakki_Siberia['YEAR'][i]) + ')'
    Leirubakki_Siberia_pole_ref.append(pole_ref)

Assign columns:

In [ ]:
Siberia['name'] = Leirubakki_Siberia['POLENAME']
Siberia['terrane'] = Leirubakki_Siberia['Terrane']
Siberia['age'] = Leirubakki_Siberia['nominal age']
Siberia['age upper'] = Leirubakki_Siberia['himagage']
Siberia['age lower'] = Leirubakki_Siberia['lomagage']
Siberia['plat'] = Leirubakki_Siberia['PLAT']
Siberia['plon'] = Leirubakki_Siberia['PLON']
Siberia['a95'] = Leirubakki_Siberia['A95']
#Siberia['f']
Siberia['slat'] = Leirubakki_Siberia['SLAT']
Siberia['slon'] = Leirubakki_Siberia['SLON']
Siberia['dec'] = Leirubakki_Siberia['DEC']
Siberia['inc'] = Leirubakki_Siberia['INC']
#Siberia['dir_a95']
Siberia['pole ref'] = Leirubakki_Siberia_pole_ref
#Siberia['age ref']
Siberia['grade'] = Leirubakki_Siberia['Grade']
#Siberia['note']
#Siberia['plateID'] = 401

Assign plateIDs:

In [ ]:
# 403 for most poles
Siberia['plateID'] = 403

# other poles
Siberia.loc[Siberia['name']=='Linok Formation','plateID'] = 401
Siberia.loc[Siberia['name']=='Kartochka Formation','plateID'] = 401
Siberia.loc[Siberia['name']=='Kitoi Cryogenian dykes','plateID'] = 401

#### Summary

In [ ]:
Siberia.sort_values('age', ascending=False, inplace=True)
Siberia = Siberia[compilation_cols]
Siberia.reset_index(inplace=True, drop=True)
Siberia

In [ ]:
Siberia.to_csv('../Output/Siberia-poles.csv',index=False)
Siberia_vpgFeatureCollection = create_vgp_FeatureCollection(Siberia)
create_vgp_gpml(Siberia_vpgFeatureCollection, '../Output/Siberia-poles.gpml')